In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a DataFrame
summary_result_path = "../../exp_results/result_summary.csv"
df = pd.read_csv(summary_result_path, sep=",")

# Calculate accuracy of 'proposed_schedulability' compared to 'simulator_schedulability'
df['accuracy'] =  df['simulator_schedulability'] == df['proposed_schedulability']

# Group the data by 'numTasks' and 'utilization' and calculate the accuracy rate
grouped_accuracy = df.groupby(['numTasks', 'utilization'])['accuracy'].mean().reset_index()

In [5]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Read the CSV file into a DataFrame
summary_result_path = "../../exp_results/result_summary.csv"
df = pd.read_csv(summary_result_path, sep=",")

# Initialize empty lists to store the results
results = []

# Get unique combinations of numTasks and utilization
combinations = df[['numTasks', 'utilization']].drop_duplicates()

In [6]:
# Loop through each combination
for i, (num_tasks, utilization) in combinations.iterrows():
    # Subset the data for the current combination
    subset = df[(df['numTasks'] == num_tasks) & (df['utilization'] == utilization)]

    # Calculate confusion matrix for the subset
    confusion = confusion_matrix(subset['simulator_schedulability'], subset['proposed_schedulability'])

    # Calculate True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN)
    if (confusion.shape == (2, 2)):
        TP = confusion[1, 1]
        TN = confusion[0, 0]
        FP = confusion[0, 1]
        FN = confusion[1, 0]
    else:
        assert confusion.shape == (1, 1)
        if subset['simulator_schedulability'].iloc[0] == False:
            TP = 0
            TN = confusion[0, 0]
            FP = 0
            FN = 0
        else:
            TP = confusion[0, 0]
            TN = 0
            FP = 0
            FN = 0
            
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall = TP / (TP + FN) if (TP + FN) != 0 else 0

    # Append the results to the list
    results.append([num_tasks, utilization, TP, TN, FP, FN, accuracy])
    # Calculate accuracy, precision, and recall for the subset

# Create a DataFrame to store the results
results_df = pd.DataFrame(results, columns=['numTasks', 'utilization', 'TP', 'TN', 'FP', 'FN', 'accuracy'])

# Save the results to a CSV file
results_df.to_csv('../../exp_results/correctness_summary.csv', index=False)

# Display the results
print(results_df)


    numTasks  utilization  TP  TN  FP  FN  accuracy
0        3.0          0.2  47   1   1   1      0.96
1        3.0          0.4  47   3   0   0      1.00
2        3.0          0.6  41   8   1   0      0.98
3        3.0          0.8  33  17   0   0      1.00
4        6.0          0.2  37   4   0   9      0.82
5        6.0          0.4  32   9   0   9      0.82
6        6.0          0.6  28  18   1   3      0.92
7        6.0          0.8  16  28   2   4      0.88
8        9.0          0.2  24  10   0  16      0.68
9        9.0          0.4  18  20   0  12      0.76
10       9.0          0.6  13  28   0   9      0.82
11       9.0          0.8   6  40   1   3      0.92
12      12.0          0.2  15  17   0  18      0.64
13      12.0          0.4  14  16   0  20      0.60
14      12.0          0.6   6  35   0   9      0.82
15      12.0          0.8   1  42   1   6      0.86
16      15.0          0.2  24   8   0  18      0.64
17      15.0          0.4  15  22   0  13      0.74
18      15.0

In [9]:
subset = df[(df['numTasks'] == 3) & (df['utilization'] == 0.2)]

# merge subsets of simulator_schedulability and proposed_schedulability
print(subset[['simulator_schedulability', 'proposed_schedulability']])


    simulator_schedulability  proposed_schedulability
0                       True                     True
1                       True                     True
2                       True                     True
3                       True                     True
4                       True                    False
5                       True                     True
6                       True                     True
7                       True                     True
8                       True                     True
9                       True                     True
10                      True                     True
11                      True                     True
12                      True                     True
13                      True                     True
14                      True                     True
15                      True                     True
16                      True                     True
17                      True